In [10]:
import os
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, Callback
from tensorflow.keras.mixed_precision import set_global_policy
import datetime
import tqdm

In [11]:
DATASET_DIR = "../FBMM/Unsplitted_Ready_Sets/set_01_class_balanced_augs_applied_splitted"  # Path to the split dataset
MODEL_SAVE_PATH = "./models/efficientnet_b0_emotion.h5"  # Where to save the best model
BATCH_SIZE = 64  # Optimal batch size for GTX 1660 Ti
IMG_SIZE = (260, 260)  # Match EfficientNetB0 input size
EPOCHS = 50  # Optimal range (Early Stopping will halt earlier if needed)
INITIAL_LR = 0.001  # Starting learning rate

In [12]:
# =============================
# 🚀 GPU CONFIGURATION
# =============================
print("\n🔍 Checking GPU Availability...")
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    print(f"✅ Using GPU: {gpus[0]}")
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)  # Allow dynamic memory growth
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')  # Force GPU usage
    except RuntimeError as e:
        print(f"⚠️ GPU Memory Growth Error: {e}")
else:
    print("❌ No GPU found! Training will be slow.")

# Enable mixed precision for better performance
print("\n🔧 Enabling Mixed Precision Training...")
set_global_policy('mixed_float16')

# Enable XLA Compilation
# print("\n⚡ Enabling XLA Optimization...")
# tf.config.optimizer.set_jit(True)


🔍 Checking GPU Availability...
✅ Using GPU: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')

🔧 Enabling Mixed Precision Training...
INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce GTX 1660 Ti, compute capability 7.5


In [14]:
# =============================
# 🚀 DATA LOADING (Using tf.data for Efficiency)
# =============================
AUTOTUNE = tf.data.experimental.AUTOTUNE

# Load dataset & store class names **before prefetching**
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    os.path.join(DATASET_DIR, "train"),
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='categorical'
)

val_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    os.path.join(DATASET_DIR, "val"),
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='categorical'
)

# ✅ Fix: Retrieve class names before prefetching
class_names = train_dataset.class_names
num_classes = len(class_names)  # Total emotion classes

# Prefetch data for better performance
train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
val_dataset = val_dataset.prefetch(buffer_size=AUTOTUNE)

Found 124012 files belonging to 7 classes.
Found 15498 files belonging to 7 classes.


In [15]:
# =============================
# 🚀 MODEL DEFINITION
# =============================
base_model = EfficientNetB0(weights="imagenet", include_top=False, input_shape=(260, 260, 3))
base_model.trainable = False  # Freeze base model

x = GlobalAveragePooling2D()(base_model.output)
x = Dense(256, activation="relu")(x)
x = Dropout(0.4)(x)
output_layer = Dense(num_classes, activation="softmax")(x)  # ✅ Fixed

model = Model(inputs=base_model.input, outputs=output_layer)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=INITIAL_LR),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

In [16]:
# =============================
# 🚀 CALLBACKS
# =============================
class CustomCallback(Callback):
    def on_epoch_begin(self, epoch, logs=None):
        # Show progress bar at the beginning of each epoch
        self.progress_bar = tqdm.tqdm(total=len(train_dataset), desc=f"Epoch {epoch+1}/{EPOCHS}", unit="batch")

        # Adjust GPU memory after 2nd epoch
        if epoch == 2:
            print("\n⚡ Adjusting GPU memory to 75% after 2 epochs.")
            for gpu in gpus:
                tf.config.experimental.set_virtual_device_configuration(
                    gpu,
                    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=int(0.75 * 6144))]  # 75% of 6GB
                )

    def on_batch_end(self, batch, logs=None):
        # Update progress bar for each batch
        self.progress_bar.update(1)

    def on_epoch_end(self, epoch, logs=None):
        # Close the progress bar at the end of each epoch
        self.progress_bar.close()
        print(f"\n🔎 Epoch {epoch+1}/{EPOCHS} - Loss: {logs['loss']:.4f}, Accuracy: {logs['accuracy']:.4f}, "
              f"Val Loss: {logs['val_loss']:.4f}, Val Accuracy: {logs['val_accuracy']:.4f}")

callbacks = [
    EarlyStopping(monitor="val_loss", patience=5, verbose=1, restore_best_weights=True),
    ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, verbose=1),
    ModelCheckpoint(MODEL_SAVE_PATH, monitor="val_accuracy", save_best_only=True, verbose=1),
    CustomCallback()  # ✅ Added custom progress bar and GPU adjustment
]

In [18]:
# =============================
# 🚀 TRAINING FUNCTION (Force GPU Execution)
# =============================
with tf.device('/GPU:0'):  # ✅ Forces TensorFlow to run on GPU
    history = model.fit(
        train_dataset,
        epochs=EPOCHS,
        validation_data=val_dataset,
        callbacks=callbacks,
        verbose=0  # ✅ Disabled default logs (handled by progress bar)
    )

# =============================
# 🚀 EVALUATE
# =============================
print("\n📊 Evaluating on test data...")
test_loss, test_accuracy = model.evaluate(val_dataset, verbose=2)
print(f"\n✅ Final Test Accuracy: {test_accuracy * 100:.2f}%")

# =============================
# 🚀 SAVE FINAL MODEL
# =============================
model.save(MODEL_SAVE_PATH)
print(f"\n✅ Model saved to: {MODEL_SAVE_PATH}")

Epoch 1/50:   0%|          | 0/1938 [00:11<?, ?batch/s]


KeyboardInterrupt: 